# ChatBot for CodeClause

#### ChatBot is an artificial intelligence program designed to simulate human-like conversations and provide assistance or information on a wide range of topics. It can answer questions, engage in interactive dialogues, and offer solutions or recommendations based on its training and knowledge. 

#### ChatBots are commonly used in customer support, virtual assistants, and other applications that require natural language processing and interaction.

# Importing Required Libraries 

In [1]:
import numpy as np
import json
import random
import pickle

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.models import load_model

from keras.optimizers import SGD

import tkinter
from tkinter import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Loading Json File

In [2]:
import json

file_location = "D:/User/Desktop/Chatbot/response.json"


with open(file_location, "r") as file:
    intents = json.load(file)

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'codeclause', 'patterns': ['About codeclause', 'What is codeclause?', 'Services offered', 'What is codeclause internship?', 'codeclause'], 'responses': ['We offer reliable, efficient delive

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'codeclause',
   'patterns': ['About codeclause',
    'What is codeclause

# Processing

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))

    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [WordNetLemmatizer().lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [5]:
print(len(words), ' Word (uniques)\n', words, '\n\n')
print(len(classes), ' Class (tags)\n', classes, '\n\n')
print(len(documents), ' Document (coubles)\n', documents)

59  Word (uniques)
 ["'s", ',', 'about', 'anyone', 'applied', 'apply', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'codeclause', 'could', 'day', 'do', 'for', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'internship', 'is', 'later', 'letter', 'me', 'my', 'next', 'nice', 'not', 'offer', 'offered', 'provide', 'receieve', 'recieved', 'see', 'service', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'what', 'when', 'where', 'will', 'you'] 


7  Class (tags)
 ['Internship', 'codeclause', 'goodbye', 'greeting', 'offer_letter', 'options', 'thanks'] 


34  Document (coubles)
 [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you',

In [7]:
output_empty = [0] *len(classes)
training = []

for doc in documents:
    bag = []
    pattern_words = doc[0]

    for word in pattern_words:
      pattern_words = WordNetLemmatizer().lemmatize(word.lower())

    for w in words:
      if w in pattern_words:
        bag.append(1)
      else:
        bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [8]:
random.shuffle(training)
training = np.array(training)
training

C:\Users\HP\AppData\Local\Temp\ipykernel_9488\1171096522.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 1, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 1, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 1])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([1, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Model Training

In [9]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
history = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=3)

Epoch 1/300
12/12 [==============================] - 1s 3ms/step - loss: 1.9540 - accuracy: 0.1471 
Epoch 2/300
12/12 [==============================] - 0s 2ms/step - loss: 1.8717 - accuracy: 0.1471
Epoch 3/300
12/12 [==============================] - 0s 2ms/step - loss: 1.8854 - accuracy: 0.2353
Epoch 4/300
12/12 [==============================] - 0s 2ms/step - loss: 1.8924 - accuracy: 0.2353
Epoch 5/300
12/12 [==============================] - 0s 2ms/step - loss: 1.7591 - accuracy: 0.2647
Epoch 6/300
12/12 [==============================] - 0s 2ms/step - loss: 1.7048 - accuracy: 0.3235
Epoch 7/300
12/12 [==============================] - 0s 2ms/step - loss: 1.6333 - accuracy: 0.3529
Epoch 8/300
12/12 [==============================] - 0s 2ms/step - loss: 1.6879 - accuracy: 0.3235
Epoch 9/300
12/12 [==============================] - 0s 3ms/step - loss: 1.5829 - accuracy: 0.3529
Epoch 10/300
12/12 [==============================] - 0s 1ms/step - loss: 1.5081 - accuracy: 0.5294
Epoch 11

In [12]:
model.save('chatbot-model.h5', history)
print('Model \"chatbot-model.h5\" Saved')

Model "chatbot-model.h5" Saved


# Testing


In [13]:
model = load_model('chatbot-model.h5')
print('Model \"chatbot-model.h5\" Loaded')

Model "chatbot-model.h5" Loaded


In [14]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [WordNetLemmatizer().lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

In [15]:
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)
  bag=[0] *len(words)

  for s in sentence_words:
      for i, w in enumerate(words):
          if w == s:
              bag[i]=1
              if show_details:
                print("found in bag: %s" % w)
  return (np.array(bag))

In [16]:
def predict_class(sentence, model):
  return_list = []
  p = bow(sentence, words, show_details = False)
  res = model.predict(np.array([p]))[0]
  error_threshhold = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > error_threshhold]
  results.sort(key = lambda x:x[1], reverse=True)

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return return_list

In [17]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
      if(i['tag'] == tag):
          result = random.choice(i['responses'])
          break
  return result

In [18]:
def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res

In [19]:
chatbot_response("Hello sir")

1/1 [==============================] - 0s 94ms/step


'Hi there, how can I help?'

In [20]:
chatbot_response("where is my offer letter")

1/1 [==============================] - 0s 21ms/step


'Offering internships and services in -> Data Analytics & Automation AI & Internet of Things (IoT) & Software Development & Web Development & App Development'

In [21]:
chatbot_response("what is this?")

1/1 [==============================] - 0s 22ms/step


'Hi there, how can I help?'

# Graphical User Interface (GUI)

In [22]:
def send():
  msg = EntryBox.get("1.0", 'end-1c').strip()
  EntryBox.delete("0.0", END)

  if msg != '':
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "You >> " + msg + '\n\n')
    ChatLog.config(foreground="#FFFFFF", font=("Cambria", 13, 'bold' ))

    res = chatbot_response(msg)
    ChatLog.insert(END, "Codeclause >>  " + res + '\n\n')

    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)

In [23]:
base = Tk()
base.title("CodeClause ChatBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="#000000",height=10, width=100, font="Arial",)
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(
    base, bd=1,
    font=("Cambria", 13, 'bold'),
    text="Send", command=send,
    width=10, height=3,
    bg="#DC143C", fg='#000000',
    activebackground="#FFFFFF"
    )

EntryBox = Text(
    base, bd=2,
    font="Cambria",
    width=29, height=2,
    bg="white",
    )

scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(relheight=0.100, relwidth=0.77, relx=0.01, rely=0.80)
SendButton.place(relx=0.79, rely=0.80, relheight=0.100, relwidth=0.20)

base.mainloop()

1/1 [==============================] - 0s 21ms/step
